In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, lit, expr, rand


# Etapa 1 - Configurar a sessão Spark e carregar os dados
spark = SparkSession.builder.appName("ex_nomes_aleatorios").master("local[*]").getOrCreate()


df_nomes = spark.read.csv('nomes_aleatorios.txt', header=False, inferSchema=True)
df_nomes.show(5)

+----------------+
|             _c0|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



In [25]:
# Etapa 2 - Renomear colunas e explorar Schema
df_nomes = df_nomes.withColumnRenamed("_c0", "Nomes")
df_nomes.printSchema()
df_nomes.show(10)

root
 |-- Nomes: string (nullable = true)

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



In [26]:
# Etapa 3 - Adicionar coluna escolaridade

df_nomes = df_nomes.withColumn(
    "Escolaridade",
    when((rand() < 0.33), lit("Fundamental"))
    .when((rand() < 0.66), lit("Médio"))
    .otherwise("Superior")
)

df_nomes.show(10)

+-----------------+------------+
|            Nomes|Escolaridade|
+-----------------+------------+
|   Frances Bennet|    Superior|
|    Jamie Russell|    Superior|
|   Edward Kistler|       Médio|
|    Sheila Maurer| Fundamental|
| Donald Golightly|    Superior|
|       David Gray|    Superior|
|      Joy Bennett|       Médio|
|      Paul Kriese|    Superior|
|Berniece Ornellas| Fundamental|
|    Brian Farrell|       Médio|
+-----------------+------------+
only showing top 10 rows



In [27]:
# Etapa 4 - Adicionar coluna país

# Listar países da América do Sul
paises = ["Brasil", "Argentina", "Uruguai", "Paraguai", "Chile", 
          "Peru", "Colômbia", "Venezuela", "Equador", "Bolívia", 
          "Suriname", "Guiana", "Guiana Francesa"]


# Criar coluna país com valores aleatórios
paises_array = ", ".join([f"'{p}'" for p in paises])
paises_expr = f"element_at(array({paises_array}), cast(rand() * {len(paises)} + 1 as int))"


# Adicionar coluna país
df_nomes = df_nomes.withColumn("País", expr(paises_expr))

df_nomes.show(10)

+-----------------+------------+---------------+
|            Nomes|Escolaridade|           País|
+-----------------+------------+---------------+
|   Frances Bennet|    Superior|      Venezuela|
|    Jamie Russell|    Superior|Guiana Francesa|
|   Edward Kistler|       Médio|      Argentina|
|    Sheila Maurer| Fundamental|      Venezuela|
| Donald Golightly|    Superior|       Paraguai|
|       David Gray|    Superior|        Equador|
|      Joy Bennett|       Médio|       Suriname|
|      Paul Kriese|    Superior|        Equador|
|Berniece Ornellas| Fundamental|Guiana Francesa|
|    Brian Farrell|       Médio|       Suriname|
+-----------------+------------+---------------+
only showing top 10 rows



In [28]:
# Etapa 5 - Adicionar coluna AnoNascimento (valores aleatórios entre 1945 e 2010)
df_nomes = df_nomes.withColumn(
    "AnoNascimento", 
    (rand() * (2010 - 1945) + 1945).cast('int')
)


df_nomes.show(10)

+-----------------+------------+---------------+-------------+
|            Nomes|Escolaridade|           País|AnoNascimento|
+-----------------+------------+---------------+-------------+
|   Frances Bennet|    Superior|      Venezuela|         1956|
|    Jamie Russell|    Superior|Guiana Francesa|         1965|
|   Edward Kistler|       Médio|      Argentina|         1963|
|    Sheila Maurer| Fundamental|      Venezuela|         2007|
| Donald Golightly|    Superior|       Paraguai|         2008|
|       David Gray|    Superior|        Equador|         1958|
|      Joy Bennett|       Médio|       Suriname|         1973|
|      Paul Kriese|    Superior|        Equador|         2000|
|Berniece Ornellas| Fundamental|Guiana Francesa|         1955|
|    Brian Farrell|       Médio|       Suriname|         1991|
+-----------------+------------+---------------+-------------+
only showing top 10 rows



In [29]:
# Etapa 6 - Filtrar pessoas nascidas nesse século
df_select = df_nomes.filter("AnoNascimento >= 2000")

df_select.show(10)

+----------------+------------+---------------+-------------+
|           Nomes|Escolaridade|           País|AnoNascimento|
+----------------+------------+---------------+-------------+
|   Sheila Maurer| Fundamental|      Venezuela|         2007|
|Donald Golightly|    Superior|       Paraguai|         2008|
|     Paul Kriese|    Superior|        Equador|         2000|
|    David Medina| Fundamental|      Argentina|         2003|
|    Rebecca Snow|       Médio|         Guiana|         2000|
| Jerry Chynoweth| Fundamental|         Guiana|         2009|
| Kenneth Rayburn| Fundamental|        Equador|         2008|
|     Lisa Baxley| Fundamental|Guiana Francesa|         2005|
|    James Barton|       Médio|      Venezuela|         2004|
|     Milton Rowe|       Médio|        Uruguai|         2001|
+----------------+------------+---------------+-------------+
only showing top 10 rows



In [30]:
# Etapa 7 - Usar Spark SQL

# Registrar o DataFrame como uma tabela temporária
df_nomes.createOrReplaceTempView("pessoas")

df_select = spark.sql("SELECT * FROM pessoas WHERE AnoNascimento >= 2000")

df_select.show(10)

+----------------+------------+---------------+-------------+
|           Nomes|Escolaridade|           País|AnoNascimento|
+----------------+------------+---------------+-------------+
|   Sheila Maurer| Fundamental|      Venezuela|         2007|
|Donald Golightly|    Superior|       Paraguai|         2008|
|     Paul Kriese|    Superior|        Equador|         2000|
|    David Medina| Fundamental|      Argentina|         2003|
|    Rebecca Snow|       Médio|         Guiana|         2000|
| Jerry Chynoweth| Fundamental|         Guiana|         2009|
| Kenneth Rayburn| Fundamental|        Equador|         2008|
|     Lisa Baxley| Fundamental|Guiana Francesa|         2005|
|    James Barton|       Médio|      Venezuela|         2004|
|     Milton Rowe|       Médio|        Uruguai|         2001|
+----------------+------------+---------------+-------------+
only showing top 10 rows



In [31]:
# Etapa 8 - Contar Millennials (1980-1994)
df_millennials = df_nomes.filter((df_nomes.AnoNascimento >= 1980) & (df_nomes.AnoNascimento <= 1994))

quantidade_millennials = df_millennials.count()

print(f"Quantidade de Millennials: {quantidade_millennials}")

Quantidade de Millennials: 2305920


In [32]:
# Etapa 9 - Contar Millennials (1980-1994) usando Spark SQL
df_millennials = spark.sql("SELECT COUNT (*) AS QuantidadeMillennials FROM pessoas WHERE AnoNascimento BETWEEN 1980 AND 1994")

print(f"Quantidade de Millennials: {df_millennials.collect()[0]['QuantidadeMillennials']}")

Quantidade de Millennials: 2305920


In [33]:
# Etapa 10 - Contar gerações por país

df_nomes = df_nomes.withColumnRenamed("País", "Pais").withColumnRenamed("Geração", "Geracao")


# Criar uma coluna para identificar a geração
df_nomes = df_nomes.withColumn(
    "Geracao",
    when((df_nomes.AnoNascimento >= 1944) & (df_nomes.AnoNascimento <= 1964), lit("Baby Boomers"))
    .when((df_nomes.AnoNascimento >= 1965) & (df_nomes.AnoNascimento <= 1979), lit("Geração X"))
    .when((df_nomes.AnoNascimento >= 1980) & (df_nomes.AnoNascimento <= 1994), lit("Millennials"))
    .otherwise(lit("Geração Z"))
)

# Registrar a tabela novamente
df_nomes.createOrReplaceTempView("pessoas")

# Contar as gerações por país
df_geracoes = spark.sql("SELECT Pais, Geracao, COUNT(*) as Quantidade FROM pessoas GROUP BY Pais, Geracao ORDER BY Pais, Geracao, Quantidade")

df_geracoes.show()


+---------+------------+----------+
|     Pais|     Geracao|Quantidade|
+---------+------------+----------+
|Argentina|Baby Boomers|    236920|
|Argentina|   Geração X|    178164|
|Argentina|   Geração Z|    177155|
|Argentina| Millennials|    176919|
|  Bolívia|Baby Boomers|    236638|
|  Bolívia|   Geração X|    177753|
|  Bolívia|   Geração Z|    177698|
|  Bolívia| Millennials|    176763|
|   Brasil|Baby Boomers|    236837|
|   Brasil|   Geração X|    177276|
|   Brasil|   Geração Z|    177275|
|   Brasil| Millennials|    178128|
|    Chile|Baby Boomers|    236832|
|    Chile|   Geração X|    177736|
|    Chile|   Geração Z|    176872|
|    Chile| Millennials|    177035|
| Colômbia|Baby Boomers|    236593|
| Colômbia|   Geração X|    176935|
| Colômbia|   Geração Z|    177578|
| Colômbia| Millennials|    177303|
+---------+------------+----------+
only showing top 20 rows

